# PyTorch quickstart for beginners

This short introduction uses [PyTorch](https://pytorch.org/) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

## Import necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
# Initialize TensorBoard
writer = SummaryWriter()

In [3]:
# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # First fully connected layer (input layer)
        self.fc1 = nn.Linear(28*28, 512)  # 28*28 pixels input, 512 outputs
        # Second fully connected layer (hidden layer)
        self.fc2 = nn.Linear(512, 256)    # 512 inputs, 256 outputs
        # Third fully connected layer (output layer)
        self.fc3 = nn.Linear(256, 10)     # 256 inputs, 10 outputs (10 classes)

    def forward(self, x):
        # Flatten the input tensor from [N, 1, 28, 28] to [N, 784] where
        # N is the batch size, and 784 is the number of pixels in MNIST images.
        x = x.view(-1, 28*28)
        # Apply ReLU activation function after first and second layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # No activation function after the last layer as we'll use CrossEntropyLoss
        x = self.fc3(x)
        return x

In [4]:
# Load and normalize the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize images
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [5]:
# Initialize the network and optimizer
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # Set the model to training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # Clear the gradients of all optimized tensors
        output = model(data)  # Forward pass
        loss = F.cross_entropy(output, target)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update parameters
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
            # Log to TensorBoard
            writer.add_scalar('training loss', loss.item(), epoch * len(train_loader) + batch_idx)

# Test function
def test(model, device, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0
    with torch.no_grad():  # No gradient computation for inference
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}'
          f' ({100. * correct / len(test_loader.dataset):.0f}%)\n')
    # Log to TensorBoard
    writer.add_scalar('test loss', test_loss, epoch)
    writer.add_scalar('test accuracy', 100. * correct / len(test_loader.dataset), epoch)

In [6]:
# Main training and testing loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(1, 11):  # Loop over the dataset multiple times (10 epochs)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.321845
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.918873
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.483751
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.296660
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.323477
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.315379
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.323688
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.176637
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.402069
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.170228

Test set: Average loss: 0.2719, Accuracy: 9238/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.166796
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.215550
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.501785
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.273661
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.140712
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.173882
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.209754
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.112197
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.079250
T

In [ ]:
writer.close()